In [1]:
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine

In [2]:
# Resaved the metacritic_movies.csv to get rid of commas in the numbers.
# Created schema for movies_t. Imported data into table just to make sure it worked, then deleted the table, and recreated
# to be an empty table.

In [3]:
# load the movies data.
csv_file = "Metacritic Reviews/metacritic_movies.csv"
raw_movies_df = pd.read_csv(csv_file)
raw_movies_df.head()

,movie_title,release_date,genre,meta_mixed,meta_negative,meta_positive,metascore,user_mixed,user_negative,user_positive,userscore
0,Anatomy of a Murder,1-Jul-59,"Drama,Mystery,Thriller,Crime",0,0,15,95,0,0,3,tbd
1,Bringing Up Baby,18-Feb-38,"Comedy,Romance,Family",0,1,16,91,1,0,2,tbd
2,After Life,12-May-99,"Drama,Fantasy",0,0,19,91,0,2,1,tbd
3,Gavagai,3-Aug-18,Drama,1,0,6,91,0,1,2,tbd
4,The Hustler,25-Sep-61,"Drama,Sport",1,0,17,90,0,0,3,tbd


In [4]:
# Reviews data cleaning
# Previewed in Excel.
# Removed lines of invalid data that appears to have wrapped incorrectly.

# Need to fix text for the apostrophes that have been convered to â€™
# Did Find/Replace using text editor.
# 
# created file Cleaned_metacritic_reviews.csv

In [5]:
# Load the reviews data.
csv_file = "Metacritic Reviews/Cleaned_metacritic_reviews.csv"
raw_reviews_df = pd.read_csv(csv_file, dtype={3:'str',4:'str'})
raw_reviews_df.head()

,critic_name,media,movie_title,review_date,individual_meta_score,text
0,Robbie Collin,The Telegraph,Jojo Rabbit,7-Oct-19,20,As satire it's a dismal dereliction of duty; a...
1,Roger Moore,NaN,Carrie Pilby,7-Oct-19,38,Carrie Pilby never rises to the level of hatef...
2,NaN,Empire,Gemini Man,7-Oct-19,40,It gives you two Will Smiths for the price of ...
3,Noel Murray,Los Angeles Times,In the Tall Grass,7-Oct-19,50,Despite the handsome Craig Wrobleski cinematog...
4,Jesse Hassenger,The A.V. Club,The King,7-Oct-19,50,It's hard to feel energized by a historical ep...


In [6]:
## Cleanup complete

In [7]:
## Addy's section

# Need to create table of movie posters urls from TheMovieDB.org
#
# Choice 1: Lookup table
#    Movie name, poster url

# Choice 2: Append a new column for poster url to movies_df

In [8]:
## write out all df's to postgresql

In [9]:
## test a movie lookup to search for a movie name, which should come back with:
#  movie name, all reviews for the movie, and movie poster url.

In [10]:
import requests

In [11]:
#url = 'https://api.themoviedb.org/3/movie/550?api_key=3d48b9204ca053957eb4375bb97a83c8'

In [12]:
#keep
config_path = 'http://api.themoviedb.org/3/configuration?api_key={key}'
KEY = '3d48b9204ca053957eb4375bb97a83c8'
url = config_path.format(key=KEY)
r = requests.get(url)
config = r.json()
# config

In [13]:
#keep
base_url = config['images']['base_url']
base_url

'http://image.tmdb.org/t/p/'

In [14]:
# i queryed one of the movie using the title
# the url query returns an id 

In [15]:
# This is what happens if there's no poster.

temp = 'http://api.themoviedb.org/3/search/movie?query="Besotted"&api_key=3d48b9204ca053957eb4375bb97a83c8'
r = requests.get(temp)
response = r.json()
response['results'][0]['poster_path']

In [16]:
# This is what happens if the movie title is not found.

temp = 'http://api.themoviedb.org/3/search/movie?query="dasgdsf"&api_key=3d48b9204ca053957eb4375bb97a83c8'
r = requests.get(temp)
response = r.json()
response['total_results']

0

In [17]:
# Reused
# base_url = 'http://image.tmdb.org/t/p/'
# size = 'w500'
# rel_path = '/mgwZvl0DwS1GIborSGlpTGomrJD.jpg'

In [18]:
# Reused 
# url = base_url + size + rel_path
# url

In [19]:
# save url to new lookup dataframe that has 2 columns: movie title, url


In [20]:
base_url = 'http://image.tmdb.org/t/p/'
size = 'w500' # We're looking for 500px sized posters

In [21]:
url_list = []
#movie_not_found_list = []
#poster_not_found_list = []
processed_count = len(raw_movies_df)
movies_not_found_count = 0
posters_not_found_count = 0
keyerrors_count = 0

keyerrors_f = open("keyerrors.txt", "w")
movies_not_found_f = open("movies_not_found.txt", "w")
posters_not_found_f = open("posters_not_found.txt", "w")

print('-----------------------------')
print('Begin Data Retreival. Rows left.')
print('-----------------------------\n')
for title in raw_movies_df.movie_title:
    #print(title)
    temp = 'http://api.themoviedb.org/3/search/movie?query="' + title + '"&api_key=3d48b9204ca053957eb4375bb97a83c8'
    #print(temp)
    try:
        r = requests.get(temp)
        response = r.json()
        url = base_url + size + response['results'][0]['poster_path']
        url_list.append({title,url})
        
        # Commmented out as this was printing out the complete list each time... 
        # print(url_list)
        
        #read json into a list
        #movie_url.append(response_json, [url])
        
        #Print a period every iteration so that we know this is still running
        processed_count -= 1
        if (processed_count % 25 == 0):
            print(processed_count, end = '')
        print('.', end = '')
        
    except KeyError:
        print(f'\n ** keyerror with {title} ** \n')
        keyerrors_count += 1
        keyerrors_f.write(title + '\n')
    
    except:
        if response['total_results'] == 0:
            #error_list.append(f'{title} not found. Skipping...')
            #movie_not_found_list.append(title)
            movies_not_found_f.write(title + '\n')
            movies_not_found_count += 1
        else:
            #print(f'No poster found for {title}.')
            #poster_not_found_list.append(title)
            posters_not_found_f.write(title + '\n')
            posters_not_found_count += 1

keyerrors_f.close()
movies_not_found_f.close()
posters_not_found_f.close()
print('\n\n-----------------------------')
print('Data Retrieval Complete')
print('-----------------------------')
print(f'Movies not found: {movies_not_found_count}')
print(f'Movies with no posters: {posters_not_found_count}')
print(f'Other errors: {keyerrors_count}\n')
print('See the files movies_not_found.txt, posters_not_found.txt, and keyerrors.txt for details.')      
    

-----------------------------
Begin Data Retreival. Rows left.
-----------------------------

....................9100.........................9075.........................9050.........................9025.........................9000.........................8975.........................8950.........................8925.........................8900.........................8875.........................8850.........................8825.........................8800.........................8775.........................8750.........................8725.........................8700.........................8675.........................8650.........................8625.........................8600.........................8575.........................8550.........................8525.........................8500.........................8475.........................8450.........................8425.........................8400.........................8375.........................8350............

.......2850.........................2825.........................2800.........................2775.........................2750.........................2725.........................2700.........................2675.........................2650.........................2625.........................2600.........................2575.........................2550.........................2525.........................2500.........................2475.........................2450.........................2425.........................2400.........................2375.........................2350.........................2325.........................2300.........................2275.........................2250.........................2225.........................2200.........................2175.........................2150.........................2125.........................2100.........................2075.........................2050.........................2025.........................2000...

In [22]:
url_list

[{'Anatomy of a Murder',
  'http://image.tmdb.org/t/p/w500/lMWWs7IskGldLfwwnVxHhffsVLw.jpg'},
 {'Bringing Up Baby',
  'http://image.tmdb.org/t/p/w500/pxgUiiP29ysQ4cuueXjiQdCphZG.jpg'},
 {'After Life',
  'http://image.tmdb.org/t/p/w500/iTJUqvnF2Q9BRt8MqpIR4Dud8d2.jpg'},
 {'Gavagai', 'http://image.tmdb.org/t/p/w500/mgwZvl0DwS1GIborSGlpTGomrJD.jpg'},
 {'The Hustler',
  'http://image.tmdb.org/t/p/w500/n2zbi9ePKNkCgguCTeurdwP40Ax.jpg'},
 {'The Circus',
  'http://image.tmdb.org/t/p/w500/hWw9HfQmd4Rn1Et4vgZQsEf5tEZ.jpg'},
 {'National Gallery',
  'http://image.tmdb.org/t/p/w500/jMp7xwS97qRz47HThNKIu2mvxQ4.jpg'},
 {'Tales of the Grim Sleeper',
  'http://image.tmdb.org/t/p/w500/7TkdWFd5J7dZtjP2suqSpLGQQUs.jpg'},
 {'The Waiting Room',
  'http://image.tmdb.org/t/p/w500/sgkewSYT4hM4mMY3HzhLk4m0IME.jpg'},
 {'Rosie', 'http://image.tmdb.org/t/p/w500/2HOd431LX9ggqSpBIJ7qyYhATyo.jpg'},
 {'Nossa Chape',
  'http://image.tmdb.org/t/p/w500/5GGuNaJECQQs2uxDeMXa3i9XpEA.jpg'},
 {'Fiddler: A Miracle of Miracles

In [23]:
url_df = pd.DataFrame(url_list, columns = ['movie_title','poster_url'])
url_df.tail(5)

,movie_title,poster_url
8730,http://image.tmdb.org/t/p/w500/vIb9kzseUun3QGo...,Wild Reeds
8731,http://image.tmdb.org/t/p/w500/eUL2USo8weZHhNM...,No Home Movie
8732,http://image.tmdb.org/t/p/w500/yRMex0csH3oI05Y...,Solas
8733,http://image.tmdb.org/t/p/w500/43ffZhMCWQhzMne...,J.T. Leroy
8734,Donnybrook,http://image.tmdb.org/t/p/w500/jcuEDKnM3uThdz7...


In [24]:
# New DataFrame with selected colum(movie_title) from movies_df
# movie_title_df = raw_movies_df[['movie_title']].copy
# movie_title__df

In [25]:
# result = pd.concat([movie_title_df, url_df],axis= , sort=True)

In [26]:
# reference
# for title in raw_movies_df.movie_title:
#     #print(title)
#     temp = 'http://api.themoviedb.org/3/search/movie?query="' + title + '"&api_key=3d48b9204ca053957eb4375bb97a83c8'
#     #print(temp)
#     r = requests.get(temp)
#     response = r.json()
#     url = base_url + size + response['results'][0]['poster_path']
#     print(url)